In [1]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:00


In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6582.3/8062.4 GB disk)


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eMPepqrjz9P581y3FL1D")
project = rf.workspace("utd-0dazj").project("utd2-hyo53")
version = project.version(5)
dataset = version.download("yolov11")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.


Extracting Dataset Version Zip to utd2-5 in yolov11:: 100%|██████████| 19164/19164 [00:01<00:00, 11244.21it/s]


In [4]:
import torch

from IPython.display import Image, clear_output  # to display images
#from utils.downloads import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.8.0+cu126 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=7bb997ac-b6eb-c048-1336-671825968484, pci_bus_id=0, pci_device_id=4, pci_domain_id=0, L2_cache_size=4MB)


In [5]:
import glob

# Inisialisasi list kosong
my_list = []

# Loop semua file .txt di folder labels
for filepath in glob.glob('/kaggle/working/utd2-5/test/labels/*.txt'):
    with open(filepath, 'r') as f:
        # Baca semua baris, strip newline, dan tambahkan ke my_list
        my_list.extend([line.strip() for line in f])

# Cek jumlah data
print(f"Total baris yang dikumpulkan: {len(my_list)}")

for i in range(0,len(my_list)):
    my_list[i]=int(my_list[i][0])
len(my_list)

count=0
for i in range(len(my_list)):
  if(my_list[i] not in [0,1,2]):
    count+=1
count

Total baris yang dikumpulkan: 719


0

In [6]:
data_yaml = """
train: /kaggle/working/utd2-5/train/images
val: /kaggle/working/utd2-5/valid/images
test: /kaggle/working/utd2-5/test/images

nc: 3
names: ['Bio', 'Rov', 'Trash']

roboflow:
  workspace: utd-0dazj
  project: utd2-hyo53
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/utd-0dazj/utd2-hyo53/dataset/5
"""

# Menyimpan file data.yaml dengan nama data_yolov11.yaml
with open('/kaggle/working/data_yolov11.yaml', 'w') as file:
    file.write(data_yaml)

print("File data_yolov11.yaml telah disimpan!")


File data_yolov11.yaml telah disimpan!


In [7]:
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s.pt

--2025-12-22 21:01:40--  https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/1c11d5fd-cb03-4cf0-9eb2-0b0f5601ef45?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-22T21%3A52%3A07Z&rscd=attachment%3B+filename%3Dyolo11s.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-22T20%3A51%3A11Z&ske=2025-12-22T21%3A52%3A07Z&sks=b&skv=2018-11-09&sig=gEHAfke%2Fg3uznJmvzs6nloeil496dFjW2R%2FcRtrYwQM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NjQzOTEwMCwibmJmIjoxNzY2NDM3MzAwLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvc

In [8]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [9]:
%cd /kaggle/working

!yolo task=detect mode=train model=/kaggle/working/yolo11s.pt\
data=/kaggle/working/data_yolov11.yaml \
epochs=200 \
imgsz=416 \
batch=160 \
plots=True \
save=True \
device=0,1 \
workers=16 \
project=/kaggle/working

/kaggle/working
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=160, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_yolov11.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nm

In [10]:
!zip -r train_yolov11.zip /kaggle/working/train


  adding: kaggle/working/train/ (stored 0%)
  adding: kaggle/working/train/results.csv (deflated 64%)
  adding: kaggle/working/train/val_batch2_labels.jpg (deflated 10%)
  adding: kaggle/working/train/BoxPR_curve.png (deflated 17%)
  adding: kaggle/working/train/train_batch2.jpg (deflated 4%)
  adding: kaggle/working/train/confusion_matrix_normalized.png (deflated 29%)
  adding: kaggle/working/train/BoxR_curve.png (deflated 12%)
  adding: kaggle/working/train/val_batch1_labels.jpg (deflated 16%)
  adding: kaggle/working/train/confusion_matrix.png (deflated 30%)
  adding: kaggle/working/train/args.yaml (deflated 53%)
  adding: kaggle/working/train/weights/ (stored 0%)
  adding: kaggle/working/train/weights/best.pt (deflated 8%)
  adding: kaggle/working/train/weights/last.pt (deflated 8%)
  adding: kaggle/working/train/train_batch8741.jpg (deflated 10%)
  adding: kaggle/working/train/train_batch1.jpg (deflated 4%)
  adding: kaggle/working/train/val_batch2_pred.jpg (deflated 10%)
  adding